**420-A58-SF - Algorithmes d'apprentissage non supervisé - Hiver 2023 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2023 Mikaël Swawola

# Evaluation formative #2

* Durée: 1 heure
* Travail individuel
* Documents autorisés
* Remettre ce notebook complété de vos réponses sur Lea

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

<h3><font color="red">Dans la cellule ci-dessous se trouvent toutes les bibliothèques nécéssaires à cette évaluation. Aucune bibliothèque supplémentaire n'est requise !</font><h3>

In [2]:
import json
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_distances
from helpers import load_sparse_csr
from scipy.sparse import csr_matrix

Pour rappel, l'archive `people.zip` contient 4 fichiers:

* **people_wiki.csv**: jeu de données consituté des pages Wikipedia de personnalités
* **people_wiki_map_index_to_word.json**: mapping entre les mots et les indices
* **people_wiki_word_count.npz**: vecteurs d'occurence des mots (word count) pour chaque document
* **people_wiki_tf_idf.npz**: vecteurs TF-IDF pour chaque document

## Chargement des données

Ici, la **représentation TF-IDF** sera utilisée.

In [3]:
# Chargement du jeu de données
wiki = pd.read_csv('../../data/people/people_wiki.csv')
wiki.index.name = 'id'

# Chargement des représentations TF-IDF
corpus = load_sparse_csr('../../data/people/people_wiki_tf_idf.npz')

# Chargement du mapping entre les mots et les indices
with open('../../data/people/people_wiki_map_index_to_word.json') as f:
     map_index_to_word = json.load(f)

## Inconvénient de la distance cosinus: tweets et artcles longs

Comme vu en cours (chapitre 02-02), la **distance cosinus** peut s'avérer désavantageuse dans le cas de documents très courts et très longs. Considérons le document suivant (tweet):

In [4]:
#+--------------------------------------------------------+
#|                                             +--------+ |
#|  One that shall not be named                | Follow | |
#|  @username                                  +--------+ |
#|                                                        |
#|  Democratic governments control law in response to     |
#|  popular act.                                          |
#|                                                        |
#|  8:05 AM - 16 May 2016                                 |
#|                                                        |
#|  Reply   Retweet (1,332)   Like (300)                  |
#|                                                        |
#+--------------------------------------------------------+

À quel point ce tweet et la page l'article Wikipedia de Barack Obama sont similaires ? La **représentation TF-IDF** de ce tweet est définie ci-dessous:

In [5]:
tweet = {'act': 3.4597778278724887,
 'control': 3.721765211295327,
 'democratic': 3.1026721743330414,
 'governments': 4.167571323949673,
 'in': 0.0009654063501214492,
 'law': 2.4538226269605703,
 'popular': 2.764478952022998,
 'response': 4.261461747058352,
 'to': 0.04694493768179923}

**Question 1 - En vous basant sur la variable `map_index_to_word`, quels sont les indices des mots du tweet ci-dessus ?**

In [6]:
# Compléter cette cellule ~ 1-2 lignes de code

In [7]:
word_indices = [map_index_to_word[word] for word in tweet.keys()]
word_indices

[547084, 546835, 547347, 546279, 547976, 547685, 547556, 546136, 547972]

**Question 2 - Obtenir la représentation TF-IF des mots du tweet présents dans la page de Barack Obama (35817). N'oubliez pas que la matrice `corpus` est une matrice creuse (sparse)!**

In [8]:
# Compléter cette cellule ~ 1-2 lignes de code

In [9]:
obama_tweet_words = corpus[35817, word_indices].toarray()
obama_tweet_words

array([[27.67822262, 14.88706085, 12.4106887 ,  0.        ,  0.02896219,
        14.72293576,  0.        , 12.78438524,  0.65722913]])

**Question 3 - Créer une variable `tweet_tf_idf`, vecteur TF-IDF complet sur tout le corpus. Utilisez l'aide suivante [https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html](scipy.sparse.csr_matrix)**

**Le tableau obtenu doit contenir des zéros partout, sauf aux positions d'indices trouvés à la question 1, et pour lesquelles les valeurs sont celles des valeurs contenues dans la variable `tweet`**

In [10]:
# Compléter cette cellule ~ 1-2 lignes de code

In [11]:
tweet_tf_idf = csr_matrix( (list(tweet.values()), ([0]*len(word_indices), word_indices)), shape=(1, corpus.shape[1]) )
tweet_tf_idf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.00096541, 0.        ,
        0.        ]])

**Question 4: Obtenir la représentation TF_IDF de la page de Barack Obama**

In [12]:
# Compléter cette cellule ~ 1 ligne de code
obama_tf_idf = corpus[35817]

**Question 5: Calculer la distance cosinus entre le tweet et la page de Barack Obama**

In [13]:
# Compléter cette cellule ~ 1 ligne de code

In [14]:
cosine_distances(obama_tf_idf, tweet_tf_idf)

array([[0.70591838]])

**Exercice 6: Comparer cette distance avec les 10 plus proches voisins trouvés à la question 2-2 de l'atelier 02-02-A2**

In [15]:
# Compléter cette cellule ~ 1-3 lignes de code

In [16]:
model_cosine = NearestNeighbors(algorithm='brute', metric='cosine').fit(corpus)
distances, indices = model_cosine.kneighbors(obama_tf_idf, n_neighbors=10)
distances

array([[0.        , 0.70313868, 0.7429819 , 0.7583584 , 0.77056123,
        0.7846775 , 0.78803907, 0.79092642, 0.7983226 , 0.79946636]])

## Fin de l'évaluation formative #2